In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

import tensorflow
tensorflow.keras.__version__
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import pandas as pd 
import datetime

In [54]:
# import csv
df = pd.read_csv("HR_Mockdata.csv")
# convert date columns to datetime
date_columns = ['hire_date', 'move_date']
for date in date_columns:
    df[date] =  pd.to_datetime(df[date], errors='coerce', dayfirst=True, 
                                                      yearfirst=False, format=None)
# find number of days between hire and move and convert to numerical value
df['delta'] =  (df['move_date'] - df['hire_date']).dt.days
# drop datetime columns
df = df.drop(columns=['move_date', 'hire_date'])
# drop columns not included in analysis
df = df.drop(columns=["sales \nfunction", 'country_home', 'country_host', 'end_date',
                       'left', 'fired_quit', '#', 'time_spend_company'])
# Drop null columns/rows
df = df.dropna(axis='columns', how='all')
df = df.dropna()
df.tail()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,Work_accident,promotion_last_5years,salary,leader_host,culture_home,culture_host,function_host,business Size_host,economic Perspective_host,buzz_bust,delta
14994,0.10,0.90,6,299,0,0,low,consistent,consensual,egalitarian,operations,united states,positive,buzz,4749.0
14995,0.11,0.77,6,247,0,0,medium,consistent,hierarquical,egalitarian,operations,united states,positive,buzz,8036.0
14996,0.50,0.83,7,302,0,0,medium,key talent,egalitarian,egalitarian,operations,united states,positive,buzz,2058.0
14997,0.90,0.94,5,236,0,0,low,consistent,hierarquical,egalitarian,commercial,united states,positive,buzz,1046.0
14998,0.90,0.86,5,257,0,0,low,consistent,consensual,egalitarian,medical,sbu,positive,buzz,1659.0


In [55]:
list(df.columns.values)

['satisfaction_level',
 'last_evaluation',
 'number_project',
 'average_montly_hours',
 'Work_accident',
 'promotion_last_5years',
 'salary',
 'leader_host',
 'culture_home',
 'culture_host',
 'function_host',
 'business Size_host',
 'economic Perspective_host',
 'buzz_bust',
 'delta']

In [56]:
# select column names that need to be converted to dummy variables
to_dummify = [
 'salary',
 'leader_host',
 'culture_home',
 'culture_host',
 'economic Perspective_host',
 'business Size_host', 
 'function_host'
 ]

In [57]:
for dummy in to_dummify:
    df = pd.concat([df.drop(dummy, axis=1), pd.get_dummies(df[dummy], drop_first=True,
                prefix=dummy)], axis=1)
df.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,Work_accident,promotion_last_5years,buzz_bust,delta,salary_low,salary_medium,...,function_host_communication,function_host_compliance,function_host_finance,function_host_human resources,function_host_legal,function_host_market access,function_host_marketing,function_host_medical,function_host_operations,function_host_regulatory affairs
0,0.75,1.00,4,216,0,0,buzz,761.0,1,0,...,0,0,0,0,0,0,1,0,0,0
1,0.89,0.85,5,266,0,0,buzz,761.0,1,0,...,0,0,0,0,0,0,1,0,0,0
2,0.41,0.50,2,128,0,0,buzz,761.0,1,0,...,0,0,0,0,0,0,1,0,0,0
3,0.45,0.49,2,144,0,0,buzz,761.0,1,0,...,0,0,0,0,0,0,1,0,0,0
4,0.10,0.82,7,265,0,0,buzz,761.0,1,0,...,0,0,0,0,0,0,1,0,0,0


In [58]:
X = df.drop("buzz_bust", axis=1)
y = df["buzz_bust"]
print(X.shape, y.shape)
X.head(1)

(13410, 34) (13410,)


,satisfaction_level,last_evaluation,number_project,average_montly_hours,Work_accident,promotion_last_5years,delta,salary_low,salary_medium,leader_host_key talent,...,function_host_communication,function_host_compliance,function_host_finance,function_host_human resources,function_host_legal,function_host_market access,function_host_marketing,function_host_medical,function_host_operations,function_host_regulatory affairs
0,0.75,1.0,4,216,0,0,761.0,1,0,1,...,0,0,0,0,0,0,1,0,0,0


In [59]:

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                    random_state=1, stratify=y)

In [60]:

# Create a StandardScater model and fit it to the training data
X_scaler = MinMaxScaler().fit(X_train)

C:\Users\h4wth\Anaconda3\envs\PythonData\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [61]:
# Transform the training and testing data using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [62]:
# Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [68]:

model = Sequential()
model.add(Dense(units=6, activation='relu', input_dim=34))
model.add(Dense(units=2, activation='softmax'))

In [69]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 6)                 210       
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 14        
Total params: 224
Trainable params: 224
Non-trainable params: 0
_________________________________________________________________


In [70]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Fit the model to the training data
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
 - 1s - loss: 0.5477 - acc: 0.7592
Epoch 2/100
 - 0s - loss: 0.5071 - acc: 0.7910
Epoch 3/100
 - 0s - loss: 0.4973 - acc: 0.7914
Epoch 4/100
 - 0s - loss: 0.4905 - acc: 0.7917
Epoch 5/100
 - 0s - loss: 0.4857 - acc: 0.7917
Epoch 6/100
 - 0s - loss: 0.4802 - acc: 0.7916
Epoch 7/100
 - 0s - loss: 0.4762 - acc: 0.7917
Epoch 8/100
 - 0s - loss: 0.4722 - acc: 0.7915
Epoch 9/100
 - 0s - loss: 0.4681 - acc: 0.7918
Epoch 10/100
 - 1s - loss: 0.4646 - acc: 0.7927
Epoch 11/100
 - 0s - loss: 0.4616 - acc: 0.7920
Epoch 12/100
 - 0s - loss: 0.4582 - acc: 0.7936
Epoch 13/100
 - 0s - loss: 0.4553 - acc: 0.7936
Epoch 14/100
 - 0s - loss: 0.4523 - acc: 0.7950
Epoch 15/100
 - 0s - loss: 0.4491 - acc: 0.7941
Epoch 16/100
 - 0s - loss: 0.4455 - acc: 0.7941
Epoch 17/100
 - 0s - loss: 0.4425 - acc: 0.7958
Epoch 18/100
 - 0s - loss: 0.4402 - acc: 0.7938
Epoch 19/100
 - 0s - loss: 0.4367 - acc: 0.7958
Epoch 20/100
 - 0s - loss: 0.4353 - acc: 0.7953
Epoch 21/100
 - 0s - loss: 0.4326 - acc: 0.7960
E

In [ ]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")